In [ ]:
import torch

In [ ]:
import numpy as np

In [ ]:
import sys
import pathlib

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import matplotlib.pyplot as plt

---

In [ ]:
sys.path.insert(0, str(pathlib.Path("../paper02/src")))

In [ ]:
# noinspection PyPackageRequirements, PyUnresolvedReferences
from datasets import get_blobs

---

In [ ]:
data, target = get_blobs()

# Scikit-Learn Model

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(100, ))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

In [ ]:
model.fit(X_train, y_train);

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

# PyTorch Model
[Source](https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values > 0)
y_test = torch.FloatTensor(y_test.values > 0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
class FeedForward(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FeedForward, self).__init__()

        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.activation = torch.nn.modules.activation.Tanh()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        hidden = self.fc1(x)
        activation = self.activation(hidden)
        output = self.fc2(activation)
        output = self.sigmoid(output)
        return output

In [ ]:
model = FeedForward(input_size=X_train.shape[1], hidden_size=100)
criterion = torch.nn.modules.loss.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model.train()
epoch = 200

for epoch in range(epoch):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred.squeeze(), y_train)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
y_pred = model(X_test)
after_train = criterion(y_pred.squeeze(), y_test)

print('Test loss after Training' , after_train.item())

In [ ]:
plt.hist(y_pred.detach().numpy(), alpha=.5)
plt.hist(y_test.detach().numpy(), alpha=.5)
plt.show()
plt.close()

In [ ]:
accuracy_score(y_pred.detach().numpy() > 0.5, y_test.detach().numpy() > 0.5)

---